### Study Correlation Plan

For the purpose of getting the HRV data, we will use the library Neurokit2 to handle the proceess to get the data short window and the full one.

### Flow of the Study

- Takes the Windowed version of the data (30 seconds, 1 minute and 2 minute)
- Calculate the HRV Metrics / Features
- Take the signal of the full length
- Take the study correlation

### HRV Metrics that we're going to use

| **Domain**     | **HRV Feature** | **Unit** | **Description**                                                                 |
|----------------|------------------|----------|----------------------------------------------------------------------------------|
| **Time**       | MeanNN           | ms       | Mean NN interval (Time it takes between each peak to peak) in milis              |
|                | SDNN             | ms       | Standard deviation of the RR intervals                                           |
|                | pNN50            | %        | NN50 count divided by the total number of all RR intervals                       |
|                | RMSSD            | ms       | Root mean square of successive RR interval differences                           |
|                | MeanHR           | bpm      | Mean heart rate                                                                  |
| **Frequency**  | LF               | ms²      | Power of low frequency band (0.04–0.15 Hz)                                       |
|                | HF               | ms²      | Power of high frequency band (0.15–0.4 Hz)                                       |
|                | LF/HF            | -        | Ratio of LF to HF                                                                |
| **Non-linear** | CSI              | -        | Cardiac sympathetic index                                                        |
|                | CVI              | -        | Cardiac vagal index                                                              |
|                | SD1              | -        | Standard deviation of Poincaré plot projection on the line perpendicular to line y=x |
|                | SD2              | -        | Standard deviation of Poincaré plot projection on the line y=x                  |


### Setup Requirements

In [123]:
# UST HRV and Normal HRV Correlation Analysis for Stress Detection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from glob import glob
import warnings
import neurokit2 as nk
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('ggplot')
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

In [124]:
import scipy 

def preprocess_ppg(signal, fs = 35):
    """ Computes the Preprocessed PPG Signal, this steps include the following:
        1. Moving Average Smoothing
        2. Bandpass Filtering
        
        Parameters:
        ----------
        signal (numpy array): 
            The PPG Signal to be preprocessed
        fs (float): 
            The Sampling Frequency of the Signal
            
        Returns:
        --------
        numpy array: 
            The Preprocessed PPG Signal
    
    """ 

    # 2. Bandpass filter to isolate the cardiac component (0.4-2.5 Hz)
    b_bp, a_bp = scipy.signal.butter(3, [0.7, 2.5], btype='band', fs=fs)
    filtered = scipy.signal.filtfilt(b_bp, a_bp, signal)
    
    return filtered

# 30 Seconds Plot Correlation

For 30 seconds window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 15 seconds (means the different between each short window is 15 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [125]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T1"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [126]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [127]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 30 seconds with stride of 15 seconds
        segment_length = 30 * sample_rate_video
        stride_length = 15 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            # rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            # rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T1 for POS


Processing s42_T1 for POS
Processing s43_T1 for POS
Processing s44_T1 for POS
Processing s45_T1 for POS
Processing s46_T1 for POS
Processing s47_T1 for POS
Processing s48_T1 for POS
Processing s49_T1 for POS
Processing s50_T1 for POS
Processing s51_T1 for POS
Processing s52_T1 for POS
Processing s53_T1 for POS
Processing s54_T1 for POS
Processing s55_T1 for POS
Processing s56_T1 for POS
Processing s41_T1 for LGI
Processing s42_T1 for LGI
Processing s43_T1 for LGI
Processing s44_T1 for LGI
Processing s45_T1 for LGI
Processing s46_T1 for LGI
Processing s47_T1 for LGI
Processing s48_T1 for LGI
Processing s49_T1 for LGI
Processing s50_T1 for LGI
Processing s51_T1 for LGI
Processing s52_T1 for LGI
Processing s53_T1 for LGI
Processing s54_T1 for LGI
Processing s55_T1 for LGI
Processing s56_T1 for LGI
Processing s41_T1 for OMIT
Processing s42_T1 for OMIT
Processing s43_T1 for OMIT
Processing s44_T1 for OMIT
Processing s45_T1 for OMIT
Processing s46_T1 for OMIT
Processing s47_T1 for OMIT
Proce

In [128]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T1': {'MeanNN': 644.4376549225466, 'SDNN': 133.62999168504808, 'RMSSD': 188.699464638323, 'pNN50': 68.2706775769929, 'LF': nan, 'HF': 0.11941876339366937, 'LF_HF': nan, 'PR': 93.23151876234611}, 's42_T1': {'MeanNN': 831.8775167875235, 'SDNN': 164.53366007170283, 'RMSSD': 216.88728939095373, 'pNN50': 73.52041635342094, 'LF': nan, 'HF': 0.12859736761535515, 'LF_HF': nan, 'PR': 72.47227098276052}, 's43_T1': {'MeanNN': 663.7810167952119, 'SDNN': 156.44343395798276, 'RMSSD': 219.37789662056684, 'pNN50': 73.85926617427674, 'LF': nan, 'HF': 0.10845917190320677, 'LF_HF': nan, 'PR': 90.4274719035342}, 's44_T1': {'MeanNN': 852.9396573849987, 'SDNN': 246.2366445962258, 'RMSSD': 318.10758027311385, 'pNN50': 86.23961795516048, 'LF': nan, 'HF': 0.11118782815874165, 'LF_HF': nan, 'PR': 70.50245482678456}, 's45_T1': {'MeanNN': 784.3447407663936, 'SDNN': 154.58060982677213, 'RMSSD': 199.7498526110061, 'pNN50': 78.00147303998395, 'LF': nan, 'HF': 0.12139238245188212, 'LF_HF': nan, 'PR': 76

### Getting the GT HRV Metrics

In [129]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T1 for ground truth
Processing s42_T1 for ground truth
Processing s43_T1 for ground truth
Processing s44_T1 for ground truth
Processing s45_T1 for ground truth
Processing s46_T1 for ground truth
Processing s47_T1 for ground truth
Processing s48_T1 for ground truth
Processing s49_T1 for ground truth
Processing s50_T1 for ground truth
Processing s51_T1 for ground truth
Processing s52_T1 for ground truth
Processing s53_T1 for ground truth
Processing s54_T1 for ground truth
Processing s55_T1 for ground truth
Processing s56_T1 for ground truth
{'s41_T1': {'MeanNN': 621.7878919860627, 'SDNN': 57.95651507086689, 'RMSSD': 58.41957650228469, 'pNN50': 17.073170731707318, 'LF': 0.03618153832803204, 'HF': 0.031889350091835855, 'LF_HF': 1.1345962907314016, 'PR': 96.49592855266613}, 's42_T1': {'MeanNN': 861.0276442307693, 'SDNN': 117.61908788954892, 'RMSSD': 142.68981949092463, 'pNN50': 48.55769230769231, 'LF': 0.004572555828700182, 'HF': 0.007710732968175658, 'LF_HF': 0.5930118248903

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [130]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [644.43765492 831.87751679 663.7810168  852.93965738 784.34474077
 800.37523609 781.2352871  840.60429819 621.47513793 653.00309192
 958.96130381 682.65918708 732.23836519 671.75975361 913.28666181
 850.7668217 ]
  GT values: [ 621.78789199  861.02764423  631.18374558  955.13034759  782.96326754
  804.16199552  774.93206522 1017.04545455  593.64652318  580.95671521
  958.75336022  770.27209052  920.50579897  690.45608108  669.24157303
  817.13755708]
POS - MeanNN: Removed 0 outliers, kept 16 subjects
Debug - POS - SDNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [133.62999169 164.53366007 156.44343396 246.2366446  154.58060983
  87.7533072   94.21991117 149.38870645 132.06946856 184.92113848
  83.44750975 141.85348803 128.01231114 150.67044139 513.34887175
 333.58513036]
  GT values: [ 57.95651507 117.61908789  45.78818

In [131]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.6896, p-value = 0.0031
  SDNN: Correlation = 0.0684, p-value = 0.8328
  RMSSD: Correlation = 0.0695, p-value = 0.8301
  pNN50: Correlation = 0.3692, p-value = 0.1593
  HF: Correlation = -0.2948, p-value = 0.2677
  PR: Correlation = 0.7709, p-value = 0.0005


Method: LGI
  MeanNN: Correlation = 0.6944, p-value = 0.0028
  SDNN: Correlation = 0.3995, p-value = 0.1763
  RMSSD: Correlation = 0.4090, p-value = 0.1653
  pNN50: Correlation = 0.4105, p-value = 0.1142
  HF: Correlation = 0.1737, p-value = 0.5199
  PR: Correlation = 0.7646, p-value = 0.0006


Method: OMIT
  MeanNN: Correlation = 0.7227, p-value = 0.0016
  SDNN: Correlation = 0.4061, p-value = 0.1686
  RMSSD: Correlation = 0.4184, p-value = 0.1548
  pNN50: Correlation = 0.4277, p-value = 0.0984
  HF: Correlation = -0.1672, p-value = 0.5360
  PR: Correlation = 0.7747, p-value = 0.0004


Method: GREEN
  MeanNN: Correlation = 0.7503, p-value = 0.0013
  SDNN: Correlation = 0.2232, p-value = 0.4636

In [132]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [133]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  PR: Correlation = 0.7709, p-value = 0.0005
  MeanNN: Correlation = 0.6896, p-value = 0.0031
  pNN50: Correlation = 0.3692, p-value = 0.1593
  HF: Correlation = -0.2948, p-value = 0.2677
  RMSSD: Correlation = 0.0695, p-value = 0.8301


Method: LGI
  PR: Correlation = 0.7646, p-value = 0.0006
  MeanNN: Correlation = 0.6944, p-value = 0.0028
  pNN50: Correlation = 0.4105, p-value = 0.1142
  RMSSD: Correlation = 0.4090, p-value = 0.1653
  SDNN: Correlation = 0.3995, p-value = 0.1763


Method: OMIT
  PR: Correlation = 0.7747, p-value = 0.0004
  MeanNN: Correlation = 0.7227, p-value = 0.0016
  pNN50: Correlation = 0.4277, p-value = 0.0984
  RMSSD: Correlation = 0.4184, p-value = 0.1548
  SDNN: Correlation = 0.4061, p-value = 0.1686


Method: GREEN
  MeanNN: Correlation = 0.7503, p-value = 0.0013
  PR: Correlation = 0.5339, p-value = 0.0332
  pNN50: Correlation = 0.3184, p-value = 0.2294
  HF: Correlation = -0.2777, p-value = 0.2977
  SD

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [134]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [135]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [136]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          16   767.734 778.075         -10.341        100.820   187.265  -207.948                8.370                  4.402       0.690  0.0031
   POS   SDNN          12   134.345  89.812          44.532         56.302   154.885   -65.820              110.037                 43.751       0.068  0.8328
   POS  RMSSD          12   182.265  96.703          85.561         66.041   215.001   -43.878              191.035                 64.906       0.069  0.8301
   POS  pNN50          16    76.206  43.047          33.159         23.652    79.517   -13.198             1753.820                 64.707       0.369  0.1593
   POS     HF          16     0.114

### Conclussion : 30 Seconds window

The study correlation within the 30 seconds rppg hrv metrics compare to the GT shows weak / moderate relation with the GT.

Using the bland-altman itself it shows one feature. The MeanNN (time it takes between each heart beat) have acceptable agreement with the reference based on your 20% threshold.

In [137]:
## Store the rPPG hrv metrics into the csv
output_path = "rest_rppg_hrv_metrics_window-30s.csv"

chrom_hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'PR' : [],
}

for subject_id in hrv_means['CHROM'].keys():
    chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
    chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
    chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
    chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])
    chrom_hrv_metrics['LF'].append(hrv_means['CHROM'][subject_id]['LF'])
    chrom_hrv_metrics['HF'].append(hrv_means['CHROM'][subject_id]['HF'])
    chrom_hrv_metrics['LF_HF'].append(hrv_means['CHROM'][subject_id]['LF_HF'])
    chrom_hrv_metrics['PR'].append(hrv_means['CHROM'][subject_id]['PR'])
    
## Convert the chrom_hrv_metrics to a DataFrame
chrom_df = pd.DataFrame(chrom_hrv_metrics)

## Add label Rest to the dataFrame
chrom_df['Label'] = 'Rest'

chrom_df.head()

## Save the DataFrame to a CSV file
chrom_df.to_csv(output_path, index=False)

---

# 1 Minute Plot Correlation

For 1 minute window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 30 seconds (means the different between each short window is 30 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [138]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T1"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [139]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [140]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'SD1': [],
    'SD2': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 30 seconds with stride of 15 seconds
        segment_length = 60 * sample_rate_video  # 30 seconds in samples
        stride_length = 30 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T1 for POS
Processing s42_T1 for POS
Processing s43_T1 for POS
Processing s44_T1 for POS
Processing s45_T1 for POS
Processing s46_T1 for POS
Processing s47_T1 for POS
Processing s48_T1 for POS
Processing s49_T1 for POS
Processing s50_T1 for POS
Processing s51_T1 for POS
Processing s52_T1 for POS
Processing s53_T1 for POS
Processing s54_T1 for POS
Processing s55_T1 for POS
Processing s56_T1 for POS
Processing s41_T1 for LGI
Processing s42_T1 for LGI
Processing s43_T1 for LGI
Processing s44_T1 for LGI
Processing s45_T1 for LGI
Processing s46_T1 for LGI
Processing s47_T1 for LGI
Processing s48_T1 for LGI
Processing s49_T1 for LGI
Processing s50_T1 for LGI
Processing s51_T1 for LGI
Processing s52_T1 for LGI
Processing s53_T1 for LGI
Processing s54_T1 for LGI
Processing s55_T1 for LGI
Processing s56_T1 for LGI
Processing s41_T1 for OMIT
Processing s42_T1 for OMIT
Processing s43_T1 for OMIT
Processing s44_T1 for OMIT
Processing s45_T1 for OMIT
Processing s46_T1 for OMIT
Proces

In [141]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T1': {'MeanNN': 642.8803690987538, 'SDNN': 138.15261993657117, 'RMSSD': 197.35889040826927, 'pNN50': 68.67728255187657, 'LF': 0.023797836954128727, 'HF': 0.1184037412875268, 'LF_HF': 0.2081340610886203, 'SD1': 140.30473582489634, 'SD2': 130.80104980266688, 'PR': 93.38384641418341}, 's42_T1': {'MeanNN': 836.2610722610723, 'SDNN': 170.83625079659163, 'RMSSD': 223.67348323602477, 'pNN50': 77.20876225224052, 'LF': 0.02108992828527502, 'HF': 0.10181098407725568, 'LF_HF': 0.20307097586041056, 'SD1': 159.25508224226274, 'SD2': 179.45994472838427, 'PR': 71.96394998269535}, 's43_T1': {'MeanNN': 664.0639435102267, 'SDNN': 154.92147091603695, 'RMSSD': 216.47830378062332, 'pNN50': 74.44478694924258, 'LF': 0.04962859659460659, 'HF': 0.08822164576258121, 'LF_HF': 0.7294541915752564, 'SD1': 153.9407332798588, 'SD2': 155.238622803071, 'PR': 90.36940524834894}, 's44_T1': {'MeanNN': 858.4420144052722, 'SDNN': 250.93035717300245, 'RMSSD': 324.51632873022004, 'pNN50': 88.28608664248591, 'LF'

### Getting the GT HRV Metrics

In [142]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T1 for ground truth
Processing s42_T1 for ground truth
Processing s43_T1 for ground truth
Processing s44_T1 for ground truth
Processing s45_T1 for ground truth
Processing s46_T1 for ground truth
Processing s47_T1 for ground truth
Processing s48_T1 for ground truth
Processing s49_T1 for ground truth
Processing s50_T1 for ground truth
Processing s51_T1 for ground truth
Processing s52_T1 for ground truth
Processing s53_T1 for ground truth
Processing s54_T1 for ground truth
Processing s55_T1 for ground truth
Processing s56_T1 for ground truth
{'s41_T1': {'MeanNN': 621.7878919860627, 'SDNN': 57.95651507086689, 'RMSSD': 58.41957650228469, 'pNN50': 17.073170731707318, 'LF': 0.03618153832803204, 'HF': 0.031889350091835855, 'LF_HF': 1.1345962907314016, 'SD1': [], 'SD2': [], 'PR': 96.49592855266613}, 's42_T1': {'MeanNN': 861.0276442307693, 'SDNN': 117.61908788954892, 'RMSSD': 142.68981949092463, 'pNN50': 48.55769230769231, 'LF': 0.004572555828700182, 'HF': 0.007710732968175658, 'L

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [143]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [ 642.8803691   836.26107226  664.06394351  858.44201441  778.67112466
  803.13843197  780.66788151  839.63012332  624.23582662  652.81443451
  956.17278168  685.95014015  734.26568483  675.90675685 1009.17216479
  853.00482144]
  GT values: [ 621.78789199  861.02764423  631.18374558  955.13034759  782.96326754
  804.16199552  774.93206522 1017.04545455  593.64652318  580.95671521
  958.75336022  770.27209052  920.50579897  690.45608108  669.24157303
  817.13755708]
POS - MeanNN: Removed 0 outliers, kept 16 subjects
Debug - POS - SDNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [138.15261994 170.8362508  154.92147092 250.93035717 153.38019921
  87.84071975  92.60073241 158.20719905 136.37201967 187.72800198
  87.28744344 147.42616537 140.26920483 158.5358361  537.65003692
 316.58504446]
  GT values: [ 57.95651507 117.61

In [144]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.5825, p-value = 0.0179
  SDNN: Correlation = 0.1365, p-value = 0.6722
  RMSSD: Correlation = 0.2723, p-value = 0.3680
  pNN50: Correlation = 0.3887, p-value = 0.1368
  LF: Correlation = 0.4287, p-value = 0.0975
  HF: Correlation = -0.2733, p-value = 0.3244
  LF_HF: Correlation = 0.0860, p-value = 0.7904
  PR: Correlation = 0.7275, p-value = 0.0014


Method: LGI
  MeanNN: Correlation = 0.6966, p-value = 0.0027
  SDNN: Correlation = 0.4221, p-value = 0.1508
  RMSSD: Correlation = 0.4170, p-value = 0.1564
  pNN50: Correlation = 0.4170, p-value = 0.1081
  LF: Correlation = 0.4082, p-value = 0.1165
  HF: Correlation = -0.0920, p-value = 0.7443
  LF_HF: Correlation = 0.2876, p-value = 0.3188
  PR: Correlation = 0.7619, p-value = 0.0006


Method: OMIT
  MeanNN: Correlation = 0.6973, p-value = 0.0027
  SDNN: Correlation = 0.4404, p-value = 0.1321
  RMSSD: Correlation = 0.4384, p-value = 0.1340
  pNN50: Correlation = 0.4430, p-value = 0.0857
  LF: Correlati

In [145]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [146]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  PR: Correlation = 0.7275, p-value = 0.0014
  MeanNN: Correlation = 0.5825, p-value = 0.0179
  LF: Correlation = 0.4287, p-value = 0.0975
  pNN50: Correlation = 0.3887, p-value = 0.1368
  HF: Correlation = -0.2733, p-value = 0.3244


Method: LGI
  PR: Correlation = 0.7619, p-value = 0.0006
  MeanNN: Correlation = 0.6966, p-value = 0.0027
  SDNN: Correlation = 0.4221, p-value = 0.1508
  pNN50: Correlation = 0.4170, p-value = 0.1081
  RMSSD: Correlation = 0.4170, p-value = 0.1564


Method: OMIT
  PR: Correlation = 0.7589, p-value = 0.0007
  MeanNN: Correlation = 0.6973, p-value = 0.0027
  pNN50: Correlation = 0.4430, p-value = 0.0857
  SDNN: Correlation = 0.4404, p-value = 0.1321
  RMSSD: Correlation = 0.4384, p-value = 0.1340


Method: GREEN
  PR: Correlation = 0.7453, p-value = 0.0014
  MeanNN: Correlation = 0.7347, p-value = 0.0018
  LF_HF: Correlation = 0.3845, p-value = 0.2430
  pNN50: Correlation = 0.3082, p-value = 0.2454
  SDN

---

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [147]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [148]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [149]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          16   774.705 778.075          -3.370        117.697   227.315  -234.056                9.170                  4.771       0.583  0.0179
   POS   SDNN          12   137.946  89.812          48.134         54.815   155.572   -59.305              113.123                 46.504       0.137  0.6722
   POS  RMSSD          13   196.188 101.271          94.918         67.119   226.472   -36.636              188.530                 82.397       0.272  0.3680
   POS  pNN50          16    77.372  43.047          34.325         23.447    80.282   -11.632             1780.075                 67.964       0.389  0.1368
   POS     LF          16     0.032

### Conclussion : 1 Minute Window

Stuff

In [150]:
## Store the rPPG hrv metrics into the csv
output_path = "rest_rppg_hrv_metrics_window-60s.csv"

chrom_hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'PR' : [],
}

for subject_id in hrv_means['CHROM'].keys():
    chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
    chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
    chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
    chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])
    chrom_hrv_metrics['LF'].append(hrv_means['CHROM'][subject_id]['LF'])
    chrom_hrv_metrics['HF'].append(hrv_means['CHROM'][subject_id]['HF'])
    chrom_hrv_metrics['LF_HF'].append(hrv_means['CHROM'][subject_id]['LF_HF'])
    chrom_hrv_metrics['PR'].append(hrv_means['CHROM'][subject_id]['PR'])
    
## Convert the chrom_hrv_metrics to a DataFrame
chrom_df = pd.DataFrame(chrom_hrv_metrics)

## Add label Rest to the dataFrame
chrom_df['Label'] = 'Rest'

chrom_df.head()

## Save the DataFrame to a CSV file
chrom_df.to_csv(output_path, index=False)

---

# 2 Minute Plot Correlation

For 2 minute window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 60 seconds (means the different between each short window is 60 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [151]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T1"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [152]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [153]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'SD1': [],
    'SD2': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 120 seconds with stride of 14 seconds
        segment_length = 120 * sample_rate_video
        stride_length = 45 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T1 for POS
Processing s42_T1 for POS
Processing s43_T1 for POS
Processing s44_T1 for POS
Processing s45_T1 for POS
Processing s46_T1 for POS
Processing s47_T1 for POS
Processing s48_T1 for POS
Processing s49_T1 for POS
Processing s50_T1 for POS
Processing s51_T1 for POS
Processing s52_T1 for POS
Processing s53_T1 for POS
Processing s54_T1 for POS
Processing s55_T1 for POS
Processing s56_T1 for POS
Processing s41_T1 for LGI
Processing s42_T1 for LGI
Processing s43_T1 for LGI
Processing s44_T1 for LGI
Processing s45_T1 for LGI
Processing s46_T1 for LGI
Processing s47_T1 for LGI
Processing s48_T1 for LGI
Processing s49_T1 for LGI
Processing s50_T1 for LGI
Processing s51_T1 for LGI
Processing s52_T1 for LGI
Processing s53_T1 for LGI
Processing s54_T1 for LGI
Processing s55_T1 for LGI
Processing s56_T1 for LGI
Processing s41_T1 for OMIT
Processing s42_T1 for OMIT
Processing s43_T1 for OMIT
Processing s44_T1 for OMIT
Processing s45_T1 for OMIT
Processing s46_T1 for OMIT
Proces

In [154]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T1': {'MeanNN': 637.2503840245776, 'SDNN': 129.91131313208575, 'RMSSD': 187.0611234433391, 'pNN50': 67.74193548387098, 'LF': 0.0337625284746812, 'HF': 0.13397170702540506, 'LF_HF': 0.25266083736858996, 'SD1': 132.62740594020096, 'SD2': 126.91317065886898, 'PR': 94.15551076213825}, 's42_T1': {'MeanNN': 838.1825317539603, 'SDNN': 167.1236098600755, 'RMSSD': 203.87511697676166, 'pNN50': 74.22077922077922, 'LF': 0.01569339831904846, 'HF': 0.06527377214488697, 'LF_HF': 0.23918648041286975, 'SD1': 144.66056945615315, 'SD2': 185.3056063127447, 'PR': 71.59671898414153}, 's43_T1': {'MeanNN': 662.163735212286, 'SDNN': 158.31764010381602, 'RMSSD': 215.60697927436593, 'pNN50': 75.28473101021638, 'LF': 0.05013002088538325, 'HF': 0.07394490427599074, 'LF_HF': 0.6751503024059008, 'SD1': 152.8807996105182, 'SD2': 163.14892393404227, 'PR': 90.614948373155}, 's44_T1': {'MeanNN': 847.3469387755102, 'SDNN': 255.25846275521477, 'RMSSD': 335.1070229383433, 'pNN50': 88.57142857142857, 'LF': 0.0

### Getting the GT HRV Metrics

In [155]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T1 for ground truth
Processing s42_T1 for ground truth
Processing s43_T1 for ground truth
Processing s44_T1 for ground truth
Processing s45_T1 for ground truth
Processing s46_T1 for ground truth
Processing s47_T1 for ground truth
Processing s48_T1 for ground truth
Processing s49_T1 for ground truth
Processing s50_T1 for ground truth
Processing s51_T1 for ground truth
Processing s52_T1 for ground truth
Processing s53_T1 for ground truth
Processing s54_T1 for ground truth
Processing s55_T1 for ground truth
Processing s56_T1 for ground truth
{'s41_T1': {'MeanNN': 621.7878919860627, 'SDNN': 57.95651507086689, 'RMSSD': 58.41957650228469, 'pNN50': 17.073170731707318, 'LF': 0.03618153832803204, 'HF': 0.031889350091835855, 'LF_HF': 1.1345962907314016, 'SD1': [], 'SD2': [], 'PR': 96.49592855266613}, 's42_T1': {'MeanNN': 861.0276442307693, 'SDNN': 117.61908788954892, 'RMSSD': 142.68981949092463, 'pNN50': 48.55769230769231, 'LF': 0.004572555828700182, 'HF': 0.007710732968175658, 'L

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [156]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [637.25038402 838.18253175 662.16373521 847.34693878 786.70300752
 804.26425604 779.32330827 850.81632653 621.16366045 646.46792921
 951.54285714 680.35151265 746.76291615 674.39530636 961.31182203
 880.10582011]
  GT values: [ 621.78789199  861.02764423  631.18374558  955.13034759  782.96326754
  804.16199552  774.93206522 1017.04545455  593.64652318  580.95671521
  958.75336022  770.27209052  920.50579897  690.45608108  669.24157303
  817.13755708]
POS - MeanNN: Removed 0 outliers, kept 16 subjects
Debug - POS - SDNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [129.91131313 167.12360986 158.3176401  255.25846276 148.94257137
  92.24767461  95.86106853 148.7833176  138.43938215 177.82528528
  88.64735139 143.20667227 156.21874051 160.4427724  445.39152754
 349.33126627]
  GT values: [ 57.95651507 117.61908789  45.78818

In [157]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.6496, p-value = 0.0065
  SDNN: Correlation = 0.1858, p-value = 0.5631
  RMSSD: Correlation = 0.0958, p-value = 0.7672
  pNN50: Correlation = 0.3432, p-value = 0.1932
  LF: Correlation = 0.3291, p-value = 0.2132
  HF: Correlation = -0.4199, p-value = 0.1054
  LF_HF: Correlation = 0.3142, p-value = 0.2958
  PR: Correlation = 0.7439, p-value = 0.0010


Method: LGI
  MeanNN: Correlation = 0.6984, p-value = 0.0026
  SDNN: Correlation = 0.4104, p-value = 0.1637
  RMSSD: Correlation = 0.3699, p-value = 0.2135
  pNN50: Correlation = 0.3990, p-value = 0.1258
  LF: Correlation = 0.4268, p-value = 0.0993
  HF: Correlation = 0.2313, p-value = 0.3888
  LF_HF: Correlation = 0.3992, p-value = 0.1574
  PR: Correlation = 0.7681, p-value = 0.0005


Method: OMIT
  MeanNN: Correlation = 0.6926, p-value = 0.0029
  SDNN: Correlation = 0.4255, p-value = 0.1472
  RMSSD: Correlation = 0.3793, p-value = 0.2012
  pNN50: Correlation = 0.4323, p-value = 0.0945
  LF: Correlatio

In [158]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [159]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  PR: Correlation = 0.7439, p-value = 0.0010
  MeanNN: Correlation = 0.6496, p-value = 0.0065
  HF: Correlation = -0.4199, p-value = 0.1054
  pNN50: Correlation = 0.3432, p-value = 0.1932
  LF: Correlation = 0.3291, p-value = 0.2132


Method: LGI
  PR: Correlation = 0.7681, p-value = 0.0005
  MeanNN: Correlation = 0.6984, p-value = 0.0026
  LF: Correlation = 0.4268, p-value = 0.0993
  SDNN: Correlation = 0.4104, p-value = 0.1637
  LF_HF: Correlation = 0.3992, p-value = 0.1574


Method: OMIT
  PR: Correlation = 0.7651, p-value = 0.0006
  MeanNN: Correlation = 0.6926, p-value = 0.0029
  LF: Correlation = 0.4515, p-value = 0.0792
  pNN50: Correlation = 0.4323, p-value = 0.0945
  SDNN: Correlation = 0.4255, p-value = 0.1472


Method: GREEN
  PR: Correlation = 0.7846, p-value = 0.0005
  MeanNN: Correlation = 0.7782, p-value = 0.0006
  LF_HF: Correlation = 0.6306, p-value = 0.0506
  HF: Correlation = -0.3899, p-value = 0.1354
  pNN50: Corr

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [160]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [161]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [162]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          16   773.010 778.075          -5.066        107.547   205.726  -215.857                8.726                  4.772       0.650  0.0065
   POS   SDNN          12   138.099  89.812          48.286         52.420   151.030   -54.457              111.857                 45.603       0.186  0.5631
   POS  RMSSD          12   182.898  96.703          86.194         64.607   212.823   -40.435              190.172                 65.662       0.096  0.7672
   POS  pNN50          16    77.556  43.047          34.508         23.872    81.298   -12.281             1807.113                 67.088       0.343  0.1932
   POS     LF          16     0.034

### Conclussion : 2 Minute Window

Stuff

In [163]:
## Store the rPPG hrv metrics into the csv
output_path = "rest_rppg_hrv_metrics_window-120s.csv"

chrom_hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'PR' : [],
}

for subject_id in hrv_means['CHROM'].keys():
    chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
    chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
    chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
    chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])
    chrom_hrv_metrics['LF'].append(hrv_means['CHROM'][subject_id]['LF'])
    chrom_hrv_metrics['HF'].append(hrv_means['CHROM'][subject_id]['HF'])
    chrom_hrv_metrics['LF_HF'].append(hrv_means['CHROM'][subject_id]['LF_HF'])
    chrom_hrv_metrics['PR'].append(hrv_means['CHROM'][subject_id]['PR'])
    
## Convert the chrom_hrv_metrics to a DataFrame
chrom_df = pd.DataFrame(chrom_hrv_metrics)

## Add label Rest to the dataFrame
chrom_df['Label'] = 'Rest'

chrom_df.head()

## Save the DataFrame to a CSV file
chrom_df.to_csv(output_path, index=False)